In [7]:
import pandas as pd

In [12]:
from google.colab import drive
drive.mount('/content/drive')
data = '/content/qm9_sample.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
df = pd.read_csv(data)
df.head()

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
0,gdb_65516,CC1(OC2CC12)C1CN1,3.11349,1.43865,1.22003,1.7661,77.75,-0.2408,0.0736,0.3144,1092.7152,0.170774,-403.079251,-403.070917,-403.069973,-403.111961,32.702
1,gdb_22429,ON=C1CC2CC(C1)O2,3.64063,1.24925,1.10175,1.4225,74.15,-0.2398,0.0075,0.2473,1118.7771,0.149112,-439.040010,-439.032537,-439.031592,-439.072026,29.847
2,gdb_22195,C[NH2+]CC(CC#N)=N[O-],1.98831,1.22992,0.84705,8.2679,83.81,-0.1509,0.0072,0.1581,1372.1477,0.145805,-435.173912,-435.163820,-435.162876,-435.209528,35.122
3,gdb_110809,COC1C2CC(C#C)C12,2.04220,1.82412,1.15961,1.1870,81.10,-0.2432,0.0479,0.2911,1111.5856,0.157994,-385.809933,-385.801095,-385.800151,-385.843329,33.447
4,gdb_47119,N=C1NC(=CCO1)C#C,3.30237,1.26351,0.93358,2.8254,79.84,-0.2166,-0.0328,0.1837,1187.7080,0.113096,-416.796690,-416.788635,-416.787691,-416.829476,30.396


In [14]:
!pip install rdkit
!pip install pubchempy
from rdkit import Chem
from rdkit.Chem import Descriptors, PandasTools, MolFromSmiles
from rdkit.ML.Descriptors import MoleculeDescriptors
import pubchempy as pcp
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 42.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=ca9a73f19405689d0d857349962ca134828e4084803d70fadbf49b77b3036d49
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


In [15]:
# Преобразование SMILES в объекты молекул RDKit
molecules = [Chem.MolFromSmiles(smiles) for smiles in df['smiles']]

# Создание списка дескрипторов
desc_list = [desc[0] for desc in Descriptors.descList]

# Вычисление значений дескрипторов для каждой молекулы
calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_list)
desc_values = [list(calc.CalcDescriptors(mol)) for mol in molecules]

# Создание нового DataFrame с дескрипторами
desc_df = pd.DataFrame(desc_values, columns=desc_list)

# Объединение с исходными DataFrame по индексу
result_df = pd.concat([df, desc_df], axis=1)

result_df.head()

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,gdb_65516,CC1(OC2CC12)C1CN1,3.11349,1.43865,1.22003,1.7661,77.75,-0.2408,0.0736,0.3144,...,0,0,0,0,0,0,0,0,0,0
1,gdb_22429,ON=C1CC2CC(C1)O2,3.64063,1.24925,1.10175,1.4225,74.15,-0.2398,0.0075,0.2473,...,0,0,0,0,0,0,0,0,0,0
2,gdb_22195,C[NH2+]CC(CC#N)=N[O-],1.98831,1.22992,0.84705,8.2679,83.81,-0.1509,0.0072,0.1581,...,0,0,0,0,0,0,0,0,0,0
3,gdb_110809,COC1C2CC(C#C)C12,2.04220,1.82412,1.15961,1.1870,81.10,-0.2432,0.0479,0.2911,...,0,0,0,1,0,0,0,0,0,0
4,gdb_47119,N=C1NC(=CCO1)C#C,3.30237,1.26351,0.93358,2.8254,79.84,-0.2166,-0.0328,0.1837,...,0,0,0,1,0,0,0,0,0,0


In [16]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 227 entries, mol_id to fr_urea
dtypes: float64(121), int64(104), object(2)
memory usage: 34.6+ MB


In [17]:
import pubchempy as pcp
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
!pip install 'mordred[full]'
from mordred import Calculator, descriptors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.9 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176720 sha256=e56deeb8423fdc45c7c677cff15ed11798b2ae1f03b5fa52fa149509029597c1
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-

In [18]:
# Создание объекта Calculator с использованием всех доступных дескрипторов из Mordred
calculator = Calculator(descriptors)

# Преобразование SMILES-нотаций в объекты молекул и сохранение в списке molecules
molecules_list = [Chem.MolFromSmiles(smiles) for smiles in df['smiles']]

# Вычисление дескрипторов для каждой молекулы с использованием калькулятора
descriptors_data = calculator.pandas(molecules_list)

# Объединение исходного DataFrame с данными о молекулах и DataFrame с вычисленными дескрипторами
merged_data = pd.concat([df, descriptors_data], axis=1)

100%|██████████| 20000/20000 [22:16<00:00, 14.96it/s]


In [19]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 1843 entries, mol_id to mZagreb2
dtypes: bool(2), float64(755), int64(323), object(763)
memory usage: 281.0+ MB


Отбор признаков

In [20]:
# датасет содержит строковые данные

In [21]:
# Удаление дубликатов
df_cleaned = merged_data.drop(columns=['mol_id'], axis=1)
df_cleaned.drop_duplicates(subset=['smiles'], inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)
print(df_cleaned)

                      smiles        A        B        C      mu  alpha  \
0          CC1(OC2CC12)C1CN1  3.11349  1.43865  1.22003  1.7661  77.75   
1           ON=C1CC2CC(C1)O2  3.64063  1.24925  1.10175  1.4225  74.15   
2      C[NH2+]CC(CC#N)=N[O-]  1.98831  1.22992  0.84705  8.2679  83.81   
3           COC1C2CC(C#C)C12  2.04220  1.82412  1.15961  1.1870  81.10   
4           N=C1NC(=CCO1)C#C  3.30237  1.26351  0.93358  2.8254  79.84   
...                      ...      ...      ...      ...     ...    ...   
19995        CC1OC2C3C1CC23O  3.91873  1.33889  1.22930  0.6101  74.07   
19996         CC(CC1COC1)C#C  4.04804  0.77994  0.69301  1.4398  84.43   
19997        Cc1c(c(no1)O)CO  2.12574  1.63393  0.96463  0.9577  67.78   
19998         O=CC1CC2CCC1O2  2.94435  1.47959  1.31805  2.0404  73.45   
19999        c1cc2c(nc1)COC2  3.76585  1.55344  1.11536  2.4484  77.56   

         homo    lumo     gap         r2  ...      SRW10     TSRW10  \
0     -0.2408  0.0736  0.3144  1092.7152

In [22]:
df_cleaned.drop(columns=['homo', 'lumo'], inplace=True)
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 1840 entries, smiles to mZagreb2
dtypes: bool(2), float64(753), int64(323), object(762)
memory usage: 280.5+ MB


In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
df_cleaned.to_csv('/content/qm9_2', index=False)